In [19]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
# import ipynb.fs.full.featuresNLTK as f_nltk

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

df_clustering_train = pd.read_csv('data/clustering_train.csv')
df_clustering_test = pd.read_csv('data/clustering_test.csv')

def agregar_clustering(df, train=True):
    df_cluster = df_clustering_train if train else df_clustering_test
    df_merge = pd.merge(df, df_cluster, on='id')
    return df_merge

# df_train_f = agregar_clustering(df_train, True)
# df_test_f = agregar_clustering(df_test, False)

titulo_train = pd.read_csv('./data/titulo_ohe_train.csv')
titulo_test = pd.read_csv('./data/titulo_ohe_test.csv')
train_DK = pd.read_csv('./data/df_train_KD.csv')
test_DK = pd.read_csv('./data/df_test_KD.csv')

df_train_f = pd.merge(df_train, titulo_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test, titulo_test, on= 'id', how= 'left')

df_train_f = pd.merge(df_train_f, train_DK, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, test_DK, on= 'id', how= 'left')

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f, df_test_f = utils.dividir_df_testeo(df_train_f, test_size=0.3)

In [20]:
df_test_f = features.features_independientes_precio(df_test_f)
df_test_f = features.features_dependientes_precio(df_test_f, df_train_f)

df_train_f = features.features_independientes_precio(df_train_f)
df_train_f = features.features_dependientes_precio(df_train_f, df_train_f)

# df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
# df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
# df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

# df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
# df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
# df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

df_train_f = features.rankings(df_train_f, df_train_f)
df_test_f = features.rankings(df_test_f, df_train_f)

In [21]:
# df_train_f, df_test_f = utils.dividir_df_testeo(df_train_f, test_size=0.3)

In [39]:
titulo_train

,id,palabra_venta,palabra_casa,palabra_departamento,palabra_san,palabra_polanco,palabra_condominio,palabra_lomas,palabra_excelente,palabra_ciudad,palabra_méxico
0,254099,0,0,0,0,0,0,0,0,0,0
1,53461,1,0,0,0,0,1,0,0,0,0
2,247984,1,1,0,0,0,0,0,0,0,0
3,209067,0,1,0,0,0,0,0,0,0,0
4,185997,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
236298,119879,0,0,0,0,0,0,0,0,0,0
236299,259178,0,1,0,0,0,1,0,0,0,0
236300,131932,0,0,0,1,0,0,0,0,0,0
236301,146867,0,1,0,0,0,0,0,0,0,0


## Selector de Features

In [22]:
from sklearn.base import BaseEstimator

class FeatureSelector(BaseEstimator):
    
    base_features = list(df_train_f.drop('precio', axis=1, errors='ignore').columns)
    
    def __init__(self, features):
        base_features = FeatureSelector.base_features

        self.features = features
        self.features_index = [i for i in range(len(base_features)) if base_features[i] in features]
        
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return x[:, self.features_index]

## LightGBM model

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.01, random_state=0)
        eval_set = [(x_eval, y_eval)]
        if self.boosting_type == 'dart':
            x_train = x
            y_train = y
            eval_set = None

        return super(LightGBMWrapper, self).fit(x_train, y_train, 
                                                early_stopping_rounds=15, eval_set=eval_set)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X)

params = {
 'bagging_fraction': 0.21587742093682755,
 'bagging_freq': 0,
 'max_bin': 388,
 'max_depth': 21,
 'min_data_in_leaf': 19,
 'min_split_gain': 0.21413040034431072,
 'num_leaves': 640,
 'reg_lambda': 9.336230074572562,
#  'bagging_fraction': 0.5079900308614034,
#  'bagging_freq': 0,
#  'max_bin': 1462,
#  'max_depth': 22,
#  'min_data_in_leaf': 58,
#  'min_split_gain': 0.6819500375795696,
#  'num_leaves': 668,
#  'reg_lambda': 48.786835826989865,
 'boosting_type': 'gbdt',
 'metric': 'mae',
 'n_jobs': 4,
 'objective': 'regression',
 'n_estimators': 1500,
}

lgb_m = Pipeline(steps=[
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'piscina', 'metroscubiertos_log',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'escomercial', 'promedio_metros_tipo_propiedad',
       'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dolar_promedio_mes',
       'dolar_promedio_ult_mes', 'es_antigua', 'antiguedad_bins_unif',
       'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 'palabra_venta',
       'palabra_casa', 'palabra_departamento', 'palabra_san',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'similares_count', 'densidad', 'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng']
        )),
    ('lightgbm', LightGBMWrapper(**params))
])

## XGBoost

In [24]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05, random_state=0)
        eval_set = [(x_eval, y_eval)]

        return super(xgb.XGBRegressor, self).fit(x_train, y_train, 
                                                early_stopping_rounds=10, eval_set=eval_set)
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)

hps = {
 'verbosity': 0,
 'objective': 'reg:squarederror',
 'eval_metric': 'mae',
 'n_estimators': 1000,
 'n_jobs': 4,
 'reg_alpha': 20.91434940058063,
 'colsample_bytree': 0.65,
 'learning_rate': 0.14,
 'max_depth': int(16.0),
#  'scale_pos_weight': 5,
#  'subsample': 0.9569019167459879
}

xgb_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'piscina', 'metroscubiertos_log',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'escomercial', 'promedio_metros_tipo_propiedad',
       'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dolar_promedio_mes',
       'dolar_promedio_ult_mes', 'es_antigua', 'antiguedad_bins_unif',
       'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 'palabra_venta',
       'palabra_casa', 'palabra_departamento', 'palabra_san',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'similares_count', 'densidad', 'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng'])),
    ('xgboost', XGBoostWrapper(**hps))
])

## RandomForest

In [25]:
from sklearn.ensemble import RandomForestRegressor

params = {'bootstrap': False,
          'max_features': 'sqrt',
          'min_samples_split': 4,
          'n_jobs': 2,
          'n_estimators': 100}

forest_m = Pipeline([
    ('feature_selector', FeatureSelector(['garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad',
       'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('random_forest', RandomForestRegressor(**params))
])

## ExtraTrees

In [26]:
from sklearn.ensemble import ExtraTreesRegressor

params = {'criterion': 'mse',
 'max_features': 'sqrt',
 'min_samples_split': 4}

extratrees_m =  Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'idf_titulo', 'idf_descripcion',
       'peso_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'antiguedad_bins_perc', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'distancia_ciudad_cara',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('extra_trees', ExtraTreesRegressor(n_estimators=50, **params))
])

## CatBoost

In [27]:
from catboost import CatBoostRegressor


class CatBoostWrapper(CatBoostRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05)
        eval_set = [(x_eval, y_eval)]

        # posiciones de features categoricas encontradas a mano
        cat_features = [0, 1, 2]
        return super(CatBoostWrapper, self).fit(x, y, cat_features=cat_features, eval_set=eval_set, early_stopping_rounds=25)


f_s = FeatureSelector(['garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cubiertos_provincia', 'mes', 
       'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad',
       'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'provincia', 'ciudad', 'tipodepropiedad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'] + ['provincia', 'ciudad', 'tipodepropiedad'])

params = {
    'od_wait': 50,
    'od_type': 'Iter',
    'learning_rate': 0.15,
    'l2_leaf_reg': 3,
    'depth': 12,
    'colsample_bylevel': 0.5,
    'border_count': 128,
    'iterations': 800,
    'silent': True,
    'eval_metric': 'MAE'
}


catboost_m = Pipeline([
    ('feature_selector', f_s),
    ('catboost', CatBoostWrapper(**params))
])

## Keras model

In [28]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.preprocessing import MinMaxScaler

def keras_modelo():    
    model = Sequential()
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=250, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=1, activation='linear'))

    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])  
    return model

keras_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'clustering_label', 'idf_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'prop_frecuente', 'top_provincia',
       'promedio_metros_totales_provincia', 'anio', 'mes',
       'trimestre', 'dias_desde_datos', 'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona','promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('scaler', MinMaxScaler()),
    ('keras', KerasRegressor(build_fn=keras_modelo, epochs=5))
])

## Stacking

In [29]:
from sklearn.preprocessing import MinMaxScaler
from vecstack import stacking
from vecstack_sk import StackingTransformer

modelos = [
#            ('keras', keras_m),
           ('xgboost', xgb_m),
           ('lightgbm', lgb_m), 
#            ('randomforest', forest_m),
#            ('extratrees', extratrees_m),
#            ('catboost', catboost_m)
          ]

stack = StackingTransformer(modelos, 
                          regression=True, verbose=2, n_folds=5)

stack.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: Pipeline]
[0]	validation_0-mae:2.19355e+06
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:1.89591e+06
[2]	validation_0-mae:1.64428e+06
[3]	validation_0-mae:1.42961e+06
[4]	validation_0-mae:1.2506e+06
[5]	validation_0-mae:1.10176e+06
[6]	validation_0-mae:978474
[7]	validation_0-mae:876935
[8]	validation_0-mae:794735
[9]	validation_0-mae:729509
[10]	validation_0-mae:677605
[11]	validation_0-mae:638448
[12]	validation_0-mae:607413
[13]	validation_0-mae:581955
[14]	validation_0-mae:563568
[15]	validation_0-mae:548212
[16]	validation_0-mae:536909
[17]	validation_0-mae:527672
[18]	validation_0-mae:520509
[19]	validation_0-mae:514146
[20]	validation_0-mae:510051
[21]	validation_0-mae:506881
[22]	validation_0-mae:504694
[23]	validation_0-mae:502781
[24]	validation_0-mae:501213
[25]	validation_0-mae:499661
[26]	validation_0-mae:499056


[91]	validation_0-mae:484460
[92]	validation_0-mae:484438
[93]	validation_0-mae:484437
[94]	validation_0-mae:484400
[95]	validation_0-mae:484372
[96]	validation_0-mae:484379
[97]	validation_0-mae:484339
[98]	validation_0-mae:484320
[99]	validation_0-mae:484340
[100]	validation_0-mae:484299
[101]	validation_0-mae:484293
[102]	validation_0-mae:484257
[103]	validation_0-mae:484223
[104]	validation_0-mae:484240
[105]	validation_0-mae:484222
[106]	validation_0-mae:484200
[107]	validation_0-mae:484204
[108]	validation_0-mae:484148
[109]	validation_0-mae:484123
[110]	validation_0-mae:484125
[111]	validation_0-mae:484134
[112]	validation_0-mae:484119
[113]	validation_0-mae:484104
[114]	validation_0-mae:484069
[115]	validation_0-mae:484033
[116]	validation_0-mae:484007
[117]	validation_0-mae:484007
[118]	validation_0-mae:483962
[119]	validation_0-mae:483943
[120]	validation_0-mae:483948
[121]	validation_0-mae:483942
[122]	validation_0-mae:483942
[123]	validation_0-mae:483937
[124]	validation_0-

[201]	validation_0-mae:494770
[202]	validation_0-mae:494792
[203]	validation_0-mae:494787
[204]	validation_0-mae:494794
[205]	validation_0-mae:494799
[206]	validation_0-mae:494802
[207]	validation_0-mae:494793
[208]	validation_0-mae:494796
[209]	validation_0-mae:494794
[210]	validation_0-mae:494770
[211]	validation_0-mae:494738
[212]	validation_0-mae:494739
[213]	validation_0-mae:494739
[214]	validation_0-mae:494731
[215]	validation_0-mae:494724
[216]	validation_0-mae:494720
[217]	validation_0-mae:494708
[218]	validation_0-mae:494722
[219]	validation_0-mae:494718
[220]	validation_0-mae:494724
[221]	validation_0-mae:494722
[222]	validation_0-mae:494732
[223]	validation_0-mae:494730
[224]	validation_0-mae:494717
[225]	validation_0-mae:494707
[226]	validation_0-mae:494686
[227]	validation_0-mae:494694
[228]	validation_0-mae:494693
[229]	validation_0-mae:494681
[230]	validation_0-mae:494682
[231]	validation_0-mae:494667
[232]	validation_0-mae:494672
[233]	validation_0-mae:494674
[234]	vali

[210]	validation_0-mae:485860
[211]	validation_0-mae:485854
[212]	validation_0-mae:485850
[213]	validation_0-mae:485845
[214]	validation_0-mae:485839
[215]	validation_0-mae:485818
[216]	validation_0-mae:485828
[217]	validation_0-mae:485820
[218]	validation_0-mae:485815
[219]	validation_0-mae:485816
[220]	validation_0-mae:485823
[221]	validation_0-mae:485824
[222]	validation_0-mae:485826
[223]	validation_0-mae:485824
[224]	validation_0-mae:485823
[225]	validation_0-mae:485817
[226]	validation_0-mae:485801
[227]	validation_0-mae:485791
[228]	validation_0-mae:485796
[229]	validation_0-mae:485793
[230]	validation_0-mae:485804
[231]	validation_0-mae:485802
[232]	validation_0-mae:485795
[233]	validation_0-mae:485808
[234]	validation_0-mae:485801
[235]	validation_0-mae:485803
[236]	validation_0-mae:485804
[237]	validation_0-mae:485801
Stopping. Best iteration:
[227]	validation_0-mae:485791

    fold  3:  [489966.00469288]
[0]	validation_0-mae:2.18591e+06
Will train until validation_0-mae hasn

[243]	validation_0-mae:501751
[244]	validation_0-mae:501755
[245]	validation_0-mae:501750
[246]	validation_0-mae:501752
[247]	validation_0-mae:501753
[248]	validation_0-mae:501750
[249]	validation_0-mae:501762
[250]	validation_0-mae:501766
Stopping. Best iteration:
[240]	validation_0-mae:501734

    fold  4:  [497453.89449323]
    ----
    MEAN:     [495892.18046637] + [3106.20689372]

estimator  1: [lightgbm: Pipeline]
[1]	valid_0's l1: 1.50844e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.39076e+06
[3]	valid_0's l1: 1.28686e+06
[4]	valid_0's l1: 1.19382e+06
[5]	valid_0's l1: 1.11313e+06
[6]	valid_0's l1: 1.04073e+06
[7]	valid_0's l1: 977537
[8]	valid_0's l1: 921562
[9]	valid_0's l1: 871910
[10]	valid_0's l1: 830496
[11]	valid_0's l1: 791950
[12]	valid_0's l1: 758434
[13]	valid_0's l1: 729260
[14]	valid_0's l1: 703219
[15]	valid_0's l1: 680412
[16]	valid_0's l1: 660444
[17]	valid_0's l1: 642722
[18]	valid_0's l1: 628470
[19]	valid_0's l1: 615931


[67]	valid_0's l1: 505519
[68]	valid_0's l1: 504814
[69]	valid_0's l1: 504521
[70]	valid_0's l1: 504239
[71]	valid_0's l1: 504222
[72]	valid_0's l1: 504151
[73]	valid_0's l1: 504051
[74]	valid_0's l1: 503502
[75]	valid_0's l1: 503752
[76]	valid_0's l1: 502953
[77]	valid_0's l1: 501789
[78]	valid_0's l1: 501546
[79]	valid_0's l1: 501200
[80]	valid_0's l1: 500857
[81]	valid_0's l1: 500563
[82]	valid_0's l1: 500856
[83]	valid_0's l1: 501219
[84]	valid_0's l1: 501015
[85]	valid_0's l1: 500593
[86]	valid_0's l1: 500415
[87]	valid_0's l1: 500204
[88]	valid_0's l1: 500168
[89]	valid_0's l1: 499813
[90]	valid_0's l1: 499332
[91]	valid_0's l1: 499013
[92]	valid_0's l1: 498655
[93]	valid_0's l1: 498410
[94]	valid_0's l1: 498497
[95]	valid_0's l1: 498806
[96]	valid_0's l1: 498783
[97]	valid_0's l1: 498987
[98]	valid_0's l1: 498636
[99]	valid_0's l1: 498541
[100]	valid_0's l1: 498684
[101]	valid_0's l1: 498552
[102]	valid_0's l1: 498144
[103]	valid_0's l1: 498258
[104]	valid_0's l1: 497973
[105]	v

    fold  2:  [481248.92471981]
[1]	valid_0's l1: 1.49304e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.37585e+06
[3]	valid_0's l1: 1.27194e+06
[4]	valid_0's l1: 1.1798e+06
[5]	valid_0's l1: 1.09933e+06
[6]	valid_0's l1: 1.02913e+06
[7]	valid_0's l1: 965389
[8]	valid_0's l1: 909804
[9]	valid_0's l1: 860020
[10]	valid_0's l1: 816338
[11]	valid_0's l1: 778372
[12]	valid_0's l1: 746180
[13]	valid_0's l1: 717980
[14]	valid_0's l1: 692289
[15]	valid_0's l1: 671124
[16]	valid_0's l1: 653246
[17]	valid_0's l1: 635939
[18]	valid_0's l1: 620270
[19]	valid_0's l1: 607955
[20]	valid_0's l1: 597161
[21]	valid_0's l1: 586938
[22]	valid_0's l1: 577780
[23]	valid_0's l1: 570481
[24]	valid_0's l1: 563757
[25]	valid_0's l1: 556254
[26]	valid_0's l1: 550978
[27]	valid_0's l1: 546582
[28]	valid_0's l1: 542320
[29]	valid_0's l1: 537995
[30]	valid_0's l1: 534679
[31]	valid_0's l1: 530851
[32]	valid_0's l1: 527629
[33]	valid_0's l1: 525481
[34]	valid_0's l1: 522767
[35

StackingTransformer(estimators=[('xgboost',
                                 Pipeline(memory=None,
                                          steps=[('feature_selector',
                                                  FeatureSelector(features=['antiguedad',
                                                                            'garages',
                                                                            'banos',
                                                                            'metroscubiertos',
                                                                            'metrostotales',
                                                                            'lat',
                                                                            'lng',
                                                                            'fecha',
                                                                            'piscina',
                                           

In [30]:
s_train = stack.transform(df_train_f.drop('precio', axis=1).values)
s_test = stack.transform(df_test_f.drop('precio', axis=1, errors='ignore').values)

Train set was detected.
Transforming...

estimator  0: [xgboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [lightgbm: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [lightgbm: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [31]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

features_stacking = ['stack01', 'stack02']

In [32]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [36]:
import xgboost as xgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
from hyperopt.pyll.base import scope
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK


features = ['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'piscina', 'metroscubiertos_log',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'escomercial', 'promedio_metros_tipo_propiedad',
       'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dolar_promedio_mes',
       'dolar_promedio_ult_mes', 'es_antigua', 'antiguedad_bins_unif',
       'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 'palabra_venta',
       'palabra_casa', 'palabra_departamento', 'palabra_san',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'similares_count', 'densidad', 'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng']


params_2nd = {
 'bagging_freq': 0,
 'max_bin': 1643,
 'max_depth': 6,
 'min_data_in_leaf': 62,
 'min_split_gain': 0.8965566444331939,
 'num_leaves': 592,
 'reg_lambda': 35.74667217035101,
#  'bagging_freq': 0
#  'max_bin': 1012,
#  'max_depth': 5,
#  'min_data_in_leaf': 40,
#  'min_split_gain': 0.2934964048344291,
#  'num_leaves': 469,
#  'reg_lambda': 36.47905035024398,
 'boosting_type': 'dart',
 'num_boost_round': 1000,
 'objective': 'regression',
 'metric': 'mae'
}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + features_stacking).values, df_train_s['precio'].values)

/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


LightGBMWrapper(bagging_freq=0, boosting_type='dart', class_weight=None,
                colsample_bytree=1.0, importance_type='split',
                learning_rate=0.1, max_bin=1643, max_depth=6, metric='mae',
                min_child_samples=20, min_child_weight=0.001,
                min_data_in_leaf=62, min_split_gain=0.8965566444331939,
                n_estimators=100, n_jobs=-1, num_boost_round=1000,
                num_leaves=592, objective='regression', random_state=None,
                reg_alpha=0.0, reg_lambda=35.74667217035101, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [37]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + features_stacking))
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
# df_test_s[['id', 'target']].to_csv('respuesta63.csv', index = False)

print(f'MAE Stacking-full: {utils.MAE(df_test_s["precio"].values, df_test_s["target"].values)}')

MAE Stacking-full: 495388.79655902204


In [38]:
# Sin DART:   MAE Stacking-full: 495768.2576267224

## Prediccion solo con features de stacking

In [ ]:
params_2nd = {'bagging_fraction': 0.8243831977099841,
 'bagging_freq': int(10.0),
 'feature_fraction': 0.9228324501365147,
 'learning_rate': 0.050664243951241736,
 'max_depth': int(3.0),
 'num_leaves': int(78.0),
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1200,
 'metric': 'mae'}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(s_train, df_train_f['precio'].values)

df_test_f['target'] = lgb_m_2nd.predict(s_test)
df_test_f[['id', 'target']].to_csv('respuesta47.csv', index = False)

## Prediccion con promedios

In [ ]:
df_test_f['target'] = s_test.T[1].T
df_test_f[['id', 'target']].to_csv('respuesta61.csv', index = False)

In [ ]:
from scipy.stats import mode

y_pred_test = mode(s_test, axis=1)[0]
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
import numpy as np

y_pred_test = np.average(s_test, axis=1)
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
from scipy.optimize import minimize

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [1] * len(s_test.T)
minimize(mae_res, x0)

In [ ]:
from scipy.optimize import minimize, differential_evolution

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [(-3, 4)] * len(s_test.T)
differential_evolution(mae_res, bounds=x0)

## Blending

In [ ]:
from sklearn.base import clone
from sklearn.model_selection import train_test_split


class BlendingTransformer():

    def __init__(self, models, model_blend, val_split=0.15, verbose=True):
        self.models = models
        self.val_split = val_split
        self.model_blend = model_blend
        self.verbose = verbose

        
    def simple_fit(self, x, y):
        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=1,
                                                            stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)
        


    def fit(self, x, y, x_test):

        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=0)
#                                                             stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)
        y_test = self._predict_second(x_test)

        x = np.concatenate((x, x_test))
        y = np.concatenate((y, y_test))
        x_cat = FeatureSelector(['provincia', 'intervalo_metros_cubiertos']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split, 
                                                            random_state=0)
#                                                             stratify=x_cat)
        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)


    def _fit_first(self, x, y):

        for i in range(len(self.models)):   
            if self.verbose:
                print(f'Fitting {i}...')
            self.models[i] = clone(self.models[i])
            self.models[i].fit(x, y)


    def _fit_second(self, x, y):
        if self.verbose:
            print(f'Fitting blender...')
        self.model_blend = clone(self.model_blend)
        self.model_blend.fit(self._predict_first(x), y)


            
    def _predict_first(self, x):
        preds = []
        for i in range(len(self.models)):
            y_pred = self.models[i].predict(x)
            preds.append(y_pred)

        return np.array(preds).T


    def _predict_second(self, x):
        return self.model_blend.predict(self._predict_first(x))


    def predict(self, x):
        return self._predict_second(x)
    
    
modelos = [
           lgb_m,
#            catboost_m,
           xgb_m,
#            forest_m
          ]

params_2nd = {'bagging_fraction': 0.8243831977099841,
 'bagging_freq': int(10.0),
 'feature_fraction': 0.9228324501365147,
 'learning_rate': 0.050664243951241736,
 'max_depth': int(3.0),
 'num_leaves': int(78.0),
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1200,
 'metric': 'mae'}

lgb_m_2nd = LightGBMWrapper(**params_2nd)

blend = BlendingTransformer(modelos, lgb_m_2nd)

### Simple blend

In [ ]:
blend.simple_fit(df_train_f.drop('precio', axis=1).values, df_train_f['precio'].values)

In [ ]:
utils.MAE(blend.predict(df_test_f.drop('precio', axis=1).values), df_test_f['precio'].values)

### Long blend

In [ ]:
blend.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values, df_test_f.drop('precio', axis=1, errors='ignore').values)

In [ ]:
df_test_s = df_test_f.copy()
df_test_s['target'] = blend.predict(df_test_s.drop('precio', axis=1, errors='ignore').values)
# utils.MAE(df_test_s['target'], df_test_s['precio'])
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
df_test_s[['id', 'target']].to_csv('respuesta63.csv', index = False)

In [ ]:
df_test_f['precio']